In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-28 23:26:26--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0.002s  

2024-06-28 23:26:26 (1.91 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [8]:
q = 'the course has already started, can I still enroll?'

In [9]:
index.fit(documents)

In [10]:
from openai import OpenAI

In [20]:
import os, getpass
api_key = getpass.getpass('Enter your OpenAI API key: ')

In [22]:
client = OpenAI(api_key=api_key)

In [23]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

'It depends on the specific circumstances and policies of the course and institution. Some courses may allow students to enroll late with permission from the instructor, while others may have strict deadlines for enrollment. It is best to contact the instructor or the department offering the course to inquire about late enrollment options.'

In [24]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [25]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [26]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [27]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [28]:
rag(query)

'To run Kafka in the terminal, you can follow these steps:\n- In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n\nIf you encounter the error "ModuleNotFoundError: No module named \'kafka.vendor.six.moves\'", it is suggested to use https://github.com/wbarnha/kafka-python-ng by running:\npip install kafka-python-ng'

In [29]:
rag('the course has already started, can I still enroll?')

'Based on the context provided, yes, you can still enroll in the course after the start date. You will still be eligible to submit homework, but be aware of deadlines for final projects. Additionally, after the course finishes, you will have access to all materials to follow at your own pace and prepare for the next cohort.'

In [30]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [31]:
from elasticsearch import Elasticsearch

In [32]:
es_client = Elasticsearch('http://localhost:9200') 

In [33]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [24]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [34]:
from tqdm.auto import tqdm

In [35]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [36]:
query = 'I just disovered the course. Can I still join it?'

In [37]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [38]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [39]:
rag(query)

"Based on the information provided in the FAQ database, you can still join the course after the start date. You are eligible to submit homework even if you don't register. However, there will be deadlines for turning in final projects, so it is recommended not to leave everything for the last minute."

In [40]:
rag("what systems are used in the data engineering zoomcamp? ")

'The systems used in the Data Engineering Zoomcamp include Git for version control, Kafka for streaming, and GitHub for course-related materials like exercise files and homework.'

In [42]:
rag('how many zoomcamps are going to be held in 2024? what are the names of these zoomcamps? ')

'As of 2024, there are 3 Zoom Camps held in a year, each for a separate course:\n1. Data-Engineering (Jan - Apr)\n2. MLOps (May - Aug)\n3. Machine Learning (Sep - Jan)'

In [43]:
rag('what kind of issues did people face in the data engineering zoomcamps?')

'The issues people faced in the data engineering Zoomcamps included problems with including files from the Mage repo in a personal copy of the Data Engineering Zoomcamp repo, such as GitHub seeing them as separate repositories.'